# 🔹 SEQ2SEQ’den LLM’e Dönüşüm Planı ( DECODER )

Bu plan, mevcut **SEQ2SEQ modelinizi** (LSTM + Attention) adım adım **GPT-style LLM**’e çevirmek için yol haritasını verir.


## 1️⃣ Encoder → Transformer Encoder

**Mevcut:** LSTM tabanlı, bidirectional, opsiyonel attention  
**Dönüşüm:**
- LSTM katmanları kaldırılır
- Yerine N adet **Transformer Encoder Block** eklenir:
  - Multi-Head Self-Attention
  - FeedForward (GELU / ReLU)
  - Residual + LayerNorm
- Opsiyonel: Pre-Norm yapısı tercih edilir



## 2️⃣ Decoder → Transformer Decoder

**Mevcut:** LSTM tabanlı, cross-attention (encoder-decoder), Gated FFN  
**Dönüşüm:**
- LSTM yerine **masked self-attention** + feedforward
- Encoder-Decoder attention → Cross-Attention (attention over encoder output)
- Gated FFN yerine klasik **FFN** veya GELU tabanlı FFN kullanılabilir
- Positional Encoding ve Residual + LayerNorm korunur



## 3️⃣ Multi-Head Attention Uyarlaması

- Mevcut `MultiHeadAttention` sınıfı kullanılabilir  
- Ufak değişikliklerle:
  - Masked attention eklenir (decoder için)
  - Causal mask uygulanır
  - Key padding mask ve batch handling optimize edilir



## 4️⃣ Positional Encoding

- Mevcut `PositionelEncoding` sınıfı doğrudan kullanılabilir  
- Transformer block girişine eklenir:  
  \[
  x = x + PE
  \]



## 5️⃣ Output / LM Head

- Decoder çıkışı → Linear Layer → Vocabulary boyutu
- Softmax → Token olasılıkları
- Örnek: `nn.Linear(hidden_dim, vocab_size)`


## 6️⃣ Model Birleştirme

- Embedding + Positional Encoding
- N adet Transformer Block (encoder veya decoder)
- Output projection (LM Head)
- Causal mask + Cross-Attention mask uygulanarak **autoregressive text generation** yapılır


## 7️⃣ Eğitim ve İnference

- Loss: CrossEntropy (token-level)
- Optimizer: Adam / AdamW
- Masking:
  - Decoder: causal mask
  - Encoder-Decoder: key padding mask
- Generation: greedy, top-k, top-p sampling



## ✅ Özet Yol Haritası

1. **Encoder** → Transformer blokları
2. **Decoder** → Masked self-attention + Cross-attention + FFN
3. **MultiHeadAttention** uyarlanır
4. **Positional Encoding** eklenir
5. **LM Head** eklenir
6. **Loss + Masking** ile eğitim yapılır
7. **Autoregressive Generation** ile inference

> Bu plan, mevcut SEQ2SEQ kodunuzu adım adım **GPT-style LLM**’e dönüştürmek için eksiksiz bir yol haritasıdır.


----
# 🛠 Adım 1: Embedding + Positional Encoding

* Mevcut AdaptiveDecoder içindeki embedding ve PositionelEncoding katmanını kullanacağız.

* Ama LSTM girişini direkt Transformer bloklarına vereceğiz.

> Bu aşamada embedding boyutunu hidden_size ile eşleştiriyoruz, böylece residual + attention uyumlu olur.

In [2]:
import torch
import torch.nn as nn
import math

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, x):
        return self.embedding(x)  # [B, T, embed_dim]

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


# 🛠 Adım 2: Multi-Head Attention (Self-Attention + Cross-Attention)

* Decoder için iki tip attention olacak:

* Masked Self-Attention → Kendi geçmiş token’larına bakar

* Encoder-Decoder Cross-Attention → Encoder çıktısına bakar

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        B, _, T, C = x.size()
        return x.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))

        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        return self.out_proj(self.combine_heads(out))


# 🛠 Adım 3: FeedForward + LayerNorm + Residual

* Transformer block’un temel taşı

* Decoder’da hem self-attention hem cross-attention çıkışına uygulanacak

In [4]:
class FeedForward(nn.Module):
    def __init__(self,embed_dim  , expansion = 4 , dp =0.1):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(embed_dim,embed_dim * expansion),
            nn.GELU(),
            nn.Linear(embed_dim * expansion , embed_dim),
            nn.Dropout(dp)
            )
    
    def forward(self,x):
        return self.net(x)

In [ ]:
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        
        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, dp=dp)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dp = nn.Dropout(dp)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        x = x + self.dp(self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask=self_mask))
        # Cross-Attention
        if enc_out is not None:
            x = x + self.dp(self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), mask=enc_mask))
        # FeedForward
        x = x + self.dp(self.ffn(self.norm3(x)))
        return x

* Bu adımla birlikte Transformer decoder’un temeli hazır.
---
## Tam decoder kodunu aşağıya bırakıyorum
---

In [6]:
import torch
import torch.nn as nn
import math

In [7]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, x):
        return self.embedding(x)  # [B, T, embed_dim

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [9]:
# -----------------------------
# 2️⃣ Multi-Head Attention
# -----------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        B, _, T, C = x.size()
        return x.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))

        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        return self.out_proj(self.combine_heads(out))

In [10]:
# -----------------------------
# 3️⃣ FeedForward
# -----------------------------
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=4, dp=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * expansion),
            nn.GELU(),
            nn.Linear(embed_dim * expansion, embed_dim),
            nn.Dropout(dp)
        )

    def forward(self, x):
        return self.net(x)

In [11]:
# -----------------------------
# 4️⃣ Transformer Decoder Block
# -----------------------------
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, dp=dp)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dp = nn.Dropout(dp)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        x = x + self.dp(self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask=self_mask))
        # Cross-Attention
        if enc_out is not None:
            x = x + self.dp(self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), mask=enc_mask))
        # FeedForward
        x = x + self.dp(self.ffn(self.norm3(x)))
        return x

In [12]:

# -----------------------------
# 5️⃣ Full Decoder Skeleton
# -----------------------------
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, num_layers=6, num_heads=8, dp=0.1, max_len=5000):
        super().__init__()
        self.embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([TransformerDecoderBlock(embed_dim, num_heads, dp) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x, enc_out, self_mask, enc_mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

* TokenEmbedding + PositionalEncoding → input token’ları vektörleştiriyor

* TransformerDecoderBlock → masked self-attention + cross-attention + feedforward + residual + layernorm

* nn.ModuleList ile N katman üst üste

* LM Head → vocab boyutuna projection

* forward fonksiyonu hem causal mask hem encoder attention için hazır

* Tek dikkat edilmesi gereken nokta:

* self_mask → decoder’ın kendi geçmiş token’larını görmesi için causal mask

* enc_mask → encoder padding mask, cross-attention sırasında kullanılır

In [ ]:
import torch
import torch.nn as nn
import math

# -----------------------------
# Positional Encoding (sin/cos)
# -----------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, embed_dim]
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# -----------------------------
# Token Embedding
# -----------------------------
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, x):
        return self.embedding(x)

# -----------------------------
# Multi-Head Attention
# -----------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        x = x.transpose(1, 2).contiguous()  # [B, T, num_heads, head_dim]
        B, T, _, _ = x.size()
        return x.view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))

        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        out = self.combine_heads(out)
        return self.out_proj(out)

# -----------------------------
# FeedForward
# -----------------------------
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=4, dp=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * expansion),
            nn.GELU(),
            nn.Dropout(dp),
            nn.Linear(embed_dim * expansion, embed_dim),
            nn.Dropout(dp)
        )

    def forward(self, x):
        return self.net(x)

# -----------------------------
# Transformer Decoder Block
# -----------------------------
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, dp=dp)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dp)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # masked self-attention (pre-norm)
        x = x + self.dropout(self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask=self_mask))
        # cross-attention (pre-norm) - enc_out: [B, T_enc, C]
        if enc_out is not None:
            x = x + self.dropout(self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), mask=enc_mask))
        # feed-forward
        x = x + self.dropout(self.ffn(self.norm3(x)))
        return x

# -----------------------------
# Helper: Causal mask builder
# -----------------------------
def build_causal_mask(seq_len, device=None, dtype=torch.uint8):
    """
    Returns a lower-triangular mask of shape [seq_len, seq_len] with 1 for allowed positions.
    mask[i, j] == 1  => query position i can attend to key position j (j <= i)
    Use the same mask for all batches/heads (it will broadcast).
    """
    if device is None:
        device = torch.device('cpu')
    mask = torch.tril(torch.ones((seq_len, seq_len), device=device, dtype=torch.uint8))
    return mask  # 1/0

# -----------------------------
# Full Transformer Decoder
# -----------------------------
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, num_layers=6, num_heads=8, dp=0.1, max_len=5000):
        super().__init__()
        self.tok_emb = TokenEmbedding(vocab_size, embed_dim)
        self.pos_enc = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([TransformerDecoderBlock(embed_dim, num_heads, dp) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, tgt_tokens, enc_out=None, tgt_mask=None, enc_mask=None):
        """
        tgt_tokens: [B, T_tgt]
        enc_out: [B, T_enc, C] or None
        tgt_mask: causal mask shape [T_tgt, T_tgt] or [B, T_tgt, T_tgt] with 1 allowed / 0 masked
        enc_mask: mask for cross-attn (1 allowed / 0 masked), broadcastable to attention scores
        returns logits: [B, T_tgt, vocab_size]
        """
        x = self.tok_emb(tgt_tokens)    # [B, T, C]
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, enc_out=enc_out, self_mask=tgt_mask, enc_mask=enc_mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

# -----------------------------
# Example usage helper
# -----------------------------
def make_causal_mask_for_batch(B, T, device):
    # returns mask [B, T, T] (1/0) that is lower triangular for each batch
    base = torch.tril(torch.ones((T, T), device=device, dtype=torch.uint8))
    return base.unsqueeze(0).expand(B, -1, -1)


---

# Şimdi Decoder yapısını daha ileriye taşıyalım.Ve elimizdeki modeli en optimum hale getirelim;
> ## Konularla alakalı daha net açıklamaları bu dosyada bulabilirsiniz 
**LLM & Transformers\LLM\Model Tasarımı(2) - LLM\Code\LLM_model_Encoder_(2).ipynb**
---

---

## Aşağıya direkt kod yapısının iyileştirilmiş halini bırakıyorum.Genel kod anlatımlarını zaten encoder bölümünde yaptık.Oradan inceleyebilirsiniz.

In [1]:
import torch
import torch.nn as nn
import math

# -----------------------------
# DropPath (Stochastic Depth)
# -----------------------------
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.0):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0.0 or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.size(0),) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

# -----------------------------
# Token Embedding + Positional Encoding
# -----------------------------
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# -----------------------------
# Multi-Head Attention
# -----------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        B, _, T, _ = x.size()
        return x.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        return self.out_proj(self.combine_heads(out))

# -----------------------------
# FeedForward (SwiGLU/MLP)
# -----------------------------
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=4, dp=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * expansion),
            nn.GELU(),
            nn.Dropout(dp),
            nn.Linear(embed_dim * expansion, embed_dim),
            nn.Dropout(dp)
        )
    def forward(self, x):
        return self.net(x)

# -----------------------------
# Modern Transformer Decoder Block
# -----------------------------
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1, drop_path=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, dp=dp)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_3 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        self_attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask=self_mask)
        x = x + self.drop_path(self.gamma_1 * self_attn_out)

        # Cross-Attention
        if enc_out is not None:
            cross_attn_out = self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), mask=enc_mask)
            x = x + self.drop_path(self.gamma_2 * cross_attn_out)

        # FeedForward
        ffn_out = self.ffn(self.norm3(x))
        x = x + self.drop_path(self.gamma_3 * ffn_out)

        return x

# -----------------------------
# Full Transformer Decoder
# -----------------------------
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, num_layers=6, num_heads=8, dp=0.1, max_len=5000, drop_path=0.1):
        super().__init__()
        self.embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([TransformerDecoderBlock(embed_dim, num_heads, dp, drop_path) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x, enc_out, self_mask, enc_mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits


---

# 🔹 Full LLM Transformer Decoder

Özellikler:

* Embed dim 1024, 12 katman (layers), 16 head

* PreNorm + LayerNorm + Residual + DropPath + LayerScale

* FeedForward genişletilmiş (expansion=8, opsiyonel SwiGLU)

* Masked Self-Attention + Cross-Attention

In [ ]:
import torch
import torch.nn as nn
import math

# -----------------------------
# DropPath (Stochastic Depth)
# -----------------------------
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.0):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0.0 or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.size(0),) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

# -----------------------------
# Token Embedding
# -----------------------------
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        return self.embedding(x)

# -----------------------------
# Positional Encoding
# -----------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# -----------------------------
# Multi-Head Attention
# -----------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=16, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        B, _, T, _ = x.size()
        return x.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        return self.out_proj(self.combine_heads(out))

# -----------------------------
# FeedForward
# -----------------------------
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=8, dp=0.1, use_swiglu=False):
        super().__init__()
        if use_swiglu:
            # SwiGLU activation
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion * 2),
                nn.SiLU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
        else:
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion),
                nn.GELU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
    def forward(self, x):
        return self.net(x)

# -----------------------------
# Transformer Decoder Block (LLM)
# -----------------------------
class TransformerDecoderBlockLLM(nn.Module):
    def __init__(self, embed_dim=1024, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, use_swiglu=False):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, expansion, dp, use_swiglu)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_3 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), self_mask)
        x = x + self.drop_path(self.gamma_1 * attn_out)

        # Cross-Attention
        if enc_out is not None:
            cross_out = self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), enc_mask)
            x = x + self.drop_path(self.gamma_2 * cross_out)

        # FeedForward
        ffn_out = self.ffn(self.norm3(x))
        x = x + self.drop_path(self.gamma_3 * ffn_out)

        return x

# -----------------------------
# Full Transformer Decoder (LLM)
# -----------------------------
class TransformerDecoderLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=1024, num_layers=12, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, max_len=5000, use_swiglu=False):
        super().__init__()
        self.embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([
            TransformerDecoderBlockLLM(embed_dim, num_heads, dp, drop_path, expansion, use_swiglu) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x, enc_out, self_mask, enc_mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

# Transformer Decoder Block (LLM) Akış Diyagramı (Markdown)
```bash

Input Tokens (`x`)
│
├─> **Token Embedding**
│    - nn.Embedding(vocab_size, embed_dim)
│
├─> **Positional Encoding**
│    - sin/cos pozisyon vektörleri eklenir
│
└─> **Decoder Layers** (N adet, ör: 12)
     │
     ├─ **LayerNorm 1**
     │    │
     │    └─> **Masked Self-Attention**
     │          - MultiHeadAttention(embed_dim, num_heads)
     │          - Masked (future tokens ignored)
     │          - Dropout
     │    │
     │    └─> **DropPath + γ₁ scaling**
     │    │
     │    └─> **Residual Add** (x + scaled attn)
     │
     ├─ **LayerNorm 2**
     │    │
     │    └─> **Cross-Attention (optional)**
     │          - MultiHeadAttention(embed_dim, num_heads)
     │          - Encoder output (`enc_out`) kullanılır
     │          - Mask optional
     │          - Dropout
     │    │
     │    └─> **DropPath + γ₂ scaling**
     │    │
     │    └─> **Residual Add**
     │
     ├─ **LayerNorm 3**
     │    │
     │    └─> **FeedForward Network (FFN)**
     │          - Linear → GELU/SwiGLU → Linear
     │          - Dropout
     │    │
     │    └─> **DropPath + γ₃ scaling**
     │    │
     │    └─> **Residual Add**
     │
     └─> **Output of Layer** (`x` updated)
│
├─> **Final LayerNorm**
│
└─> **LM Head**
     - Linear(embed_dim → vocab_size)
     - Output logits for next token prediction


## Açıklama:

* Input Tokens: Decoder’a gelen token ID’leri.

* Embedding + Positional Encoding: Token vektörleri ve pozisyon bilgisi.

* Masked Self-Attention: Decoder’ın kendi önceki tokenlerini dikkate alarak dikkat mekanizması.

* Cross-Attention: Encoder çıkışı varsa ona bakarak dikkat (opsiyonel).

* FeedForward: Lineer + aktivasyon + dropout katmanları.

* DropPath + γ scaling + Residual: Her blokta stochastic depth ve learnable scaling uygulanıyor.

* Çıkış: Block sonrası güncellenmiş token temsil vektörleri.